<a href="https://colab.research.google.com/github/Niquecloud/Infrastructure-provisioning-using-terrafom/blob/main/Chatbot_cs_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()  # Upload CSV file from your computer



Saving Customer-Support chatbot training data.csv to Customer-Support chatbot training data.csv


In [2]:
import pandas as pd

# Load the CSV
df = pd.read_csv('Customer-Support chatbot training data.csv')
print(df.head())
print(df.columns)

                                   query  \
0           My order hasn't arrived yet.   
1          I received a damaged product.   
2              I need to return an item.   
3  I want to change my shipping address.   
4       I have a question about my bill.   

                                            response  
0  We apologize for the inconvenience. Can you pl...  
1  We apologize for the inconvenience. Can you pl...  
2  Certainly. Please provide your order number an...  
3  No problem. Can you please provide your order ...  
4  We'd be happy to help. Can you please provide ...  
Index(['query', 'response'], dtype='object')


In [3]:
import json

# Convert to JSONL format (required for training GPT)
with open('train_data.jsonl', 'w') as f:
    for idx, row in df.iterrows():
        json.dump({"prompt": f"Customer: {row['query']}\nSupport:",
                   "completion": f" {row['response']}"}, f)
        f.write("\n")

In [4]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Adjust token embeddings if tokenizer adds new tokens (optional here)
model.resize_token_embeddings(len(tokenizer))



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Embedding(50257, 768)

In [5]:
from transformers import GPT2Tokenizer
from datasets import Dataset

# Load tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Fix: set pad_token = eos_token (so padding works without errors)
tokenizer.pad_token = tokenizer.eos_token

# Convert your pandas DataFrame into a HuggingFace Dataset
dataset = Dataset.from_pandas(df)

# Tokenization function
def tokenize_function(example):
    tokens = tokenizer(
        f"Customer: {example['query']}\nSupport: {example['response']}",
        truncation=True,
        padding="max_length",
        max_length=128
    )
    # Add labels (needed for Trainer to compute loss)
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

# Apply tokenization
tokenized_dataset = dataset.map(tokenize_function, batched=False)





Map:   0%|          | 0/74 [00:00<?, ? examples/s]

In [6]:
import os
os.environ["WANDB_DISABLED"] = "true"   # ✅ Disable W&B logging

from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    num_train_epochs=5,              # Number of epochs (loops over dataset)
    per_device_train_batch_size=1,   # Small batch size (works better on limited GPU)
    save_steps=50,                   # Save model every 50 steps
    save_total_limit=2,              # Keep only last 2 checkpoints
    logging_steps=10,                # Log training progress every 10 steps
)

trainer = Trainer(
    model=model,                      # The GPT-2 model you loaded
    args=training_args,
    train_dataset=tokenized_dataset   # Your prepared dataset
)

trainer.train()




Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
10,1.911800
20,0.700500
30,0.545300
40,0.429900
50,0.442200
60,0.507900
70,0.388500
80,0.372600
90,0.275600
100,0.279700


TrainOutput(global_step=370, training_loss=0.3003018717508058, metrics={'train_runtime': 222.8035, 'train_samples_per_second': 1.661, 'train_steps_per_second': 1.661, 'total_flos': 24169512960000.0, 'train_loss': 0.3003018717508058, 'epoch': 5.0})

In [7]:
def generate_response(query, max_length=150):
    prompt = f"Customer: {query}\nSupport:"
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(model.device)

    output_ids = model.generate(
        input_ids,
        max_length=max_length,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        top_p=0.9,
        temperature=0.8,
        early_stopping=True
    )

    response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    response = response.replace(prompt, "").strip()
    return response



In [8]:
query ="My order hasn't arrived yet."
response = generate_response(query)
print("Bot:", response)



The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Bot: We apologize for the inconvenience. Can you please provide your order number so we can resend the receipt and provide you with the item name or SKU you're interested in?


In [14]:
query ="How long does shipping take?."
response = generate_response(query)
print("Bot:", response)

Bot: We'd be happy to provide an estimate. Can you please provide the product name or SKU so we can check the shipping time for you?
